# 1. Importamos

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torch 
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 2. Procesamiento de datos

In [3]:
class DigitRecognizerDataset(Dataset):
    # train Indica si el dataset es de entrenamiento o de prueba.
    # Si train=True, la clase separa labels (etiquetas) de las imágenes.
    # Si train=False, significa que son datos de prueba y no hay etiquetas, solo imágenes
    
    def __init__(self, csv_file,transform=None,train=True):
        self.data=pd.read_csv(csv_file)
        self.transform = transform
        self.train = train
        if self.train:
            self.labels=self.data['label'].values
            self.images = self.data.drop(columns=['label']).values

        else:
            self.labels = None
            self.images = self.data.values

        # Reorganizamos las imágenes y las preparamos para la red neuronal
        self.images = (
            self.images
            .reshape(-1, 28, 28, 1)  # Cambia la forma: número de imágenes x 28 x 28 x 1 canal (blanco y negro)
            .astype(np.float32)        # Convierte los valores de píxeles a float32 para cálculos más precisos
            / 255.0                    # Normaliza los valores a rango [0,1] para que la red entrene mejor
        )

    # __len__ define cuántos elementos tiene el dataset.
    # Esto es necesario para que PyTorch sepa cuántos ejemplos hay y pueda iterar sobre ellos.
    def __len__(self):
        return len(self.images)  # Devuelve el número de imágenes en el dataset

    # __getitem__ define cómo obtener un solo ejemplo del dataset por su índice idx.
    # Esto es esencial para el DataLoader, que pide ejemplos de uno en uno o en batch.
    def __getitem__(self, idx):
        image = self.images[idx]  # Obtiene la imagen en la posición idx

        # Aplica transformaciones si se han definido (como normalización o data augmentation)
        if self.transform:
            image = self.transform(image)

        # Si estamos en modo entrenamiento, devolvemos también la etiqueta correspondiente
        if self.train:
            label = self.labels[idx]  # Obtiene la etiqueta para esa imagen
            return image, label      # Devuelve un tuple (imagen, etiqueta)
    
        # Si no estamos entrenando (por ejemplo test set), solo devolvemos la imagen
        return image


In [4]:
# Definimos las transformaciones para las imágenes de entrenamiento
# Compose permite encadenar varias transformaciones
train_transform = transforms.Compose([
    transforms.ToPILImage(),         # Convierte la imagen de numpy array a objeto PIL para poder aplicar transformaciones
    transforms.RandomRotation(10),   # Rota la imagen aleatoriamente hasta 10 grados (data augmentation)
    transforms.ToTensor(),           # Convierte la imagen PIL a tensor de PyTorch (valores entre 0 y 1)
    transforms.Normalize((0.5,), (0.5,))  # Normaliza los valores del tensor para que tengan media 0.5 y desviación 0.5
])

# Transformaciones para las imágenes de test
# Aquí no aplicamos data augmentation, solo convertimos a tensor y normalizamos
test_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

# Creamos el dataset de entrenamiento usando nuestra clase personalizada
# Le pasamos el CSV con las imágenes, las transformaciones y le indicamos que es train=True
train_dataset = DigitRecognizerDataset('/kaggle/input/digit-recognizer/train.csv',
                                       transform=train_transform,
                                       train=True)

# Creamos el dataset de test
# Aquí train=False porque no tenemos labels y no aplicamos data augmentation
test_dataset = DigitRecognizerDataset('/kaggle/input/digit-recognizer/test.csv',
                                      transform=test_transform,
                                      train=False)

# Creamos el DataLoader de entrenamiento
# batch_size=64 -> cada batch tendrá 64 imágenes
# shuffle=True -> mezcla las imágenes en cada epoch para mejorar el entrenamiento
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)

# DataLoader para test
# shuffle=False -> no mezclamos, mantenemos el orden de las imágenes
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)


# definimos modelo

In [5]:
# Definimos la clase de la CNN
class SimpleCNN(nn.Module):
    
    # __init__ se ejecuta cuando creamos una instancia del modelo
    # Aquí definimos todas las capas que va a usar la red
    def __init__(self, num_classes=10):
        super(SimpleCNN, self).__init__()  # Llama al constructor de nn.Module
        
        # Primera capa convolucional: 1 canal de entrada (grayscale), 16 filtros, kernel 3x3
        self.conv1 = nn.Conv2d(1, 16, kernel_size=3, stride=1, padding=1)
        
        # Segunda capa convolucional: 16 canales de entrada, 32 filtros
        self.conv2 = nn.Conv2d(16, 32, kernel_size=3, stride=1, padding=1)
        
        # Capa de max pooling: reduce el tamaño de la imagen a la mitad
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        
        # Capa totalmente conectada: de 32*7*7 neuronas a 128
        self.fc1 = nn.Linear(32 * 7 * 7, 128)
        
        # Capa de salida: de 128 neuronas a num_classes (10 dígitos)
        self.fc2 = nn.Linear(128, num_classes)
        
        # Dropout: apaga aleatoriamente el 25% de neuronas para evitar overfitting
        self.dropout = nn.Dropout(0.25)

    # forward define cómo pasan los datos por la red
    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))  # conv1 -> ReLU -> pooling
        x = self.pool(F.relu(self.conv2(x)))  # conv2 -> ReLU -> pooling
        x = x.view(-1, 32 * 7 * 7)            # aplana la imagen para la capa densa
        x = F.relu(self.fc1(x))               # fc1 + ReLU
        x = self.dropout(x)                    # aplica dropout
        x = self.fc2(x)                        # capa de salida
        return x

# Creamos el modelo y lo movemos a la GPU si está disponible
model = SimpleCNN().to(device)

# Definimos la función de pérdida (cross entropy, típica para clasificación)
criterion = nn.CrossEntropyLoss()

# Definimos el optimizador (Adam) con learning rate de 0.001
optimizer = optim.Adam(model.parameters(), lr=0.001)


## entrenamos modelo

In [6]:
def train_model(epochs=50):
    model.train()
    for epoch in range(epochs):
        running_loss=0.0
        for i,(images,labels) in enumerate(train_loader):
            images,labels = images.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs=model(images)
            loss=criterion(outputs,labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            if i % 200 == 199:
                print(f"[Epoch {epoch + 1}, Batch {i + 1}] Loss: {running_loss / 200:.3f}")
                running_loss = 0.0

In [7]:
def evaluate_model():
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    print(f"Validation Accuracy: {100 * correct / total:.2f}%")

In [8]:
train_model(epochs=10)
evaluate_model()

[Epoch 1, Batch 200] Loss: 0.701
[Epoch 1, Batch 400] Loss: 0.207
[Epoch 1, Batch 600] Loss: 0.160
[Epoch 2, Batch 200] Loss: 0.109
[Epoch 2, Batch 400] Loss: 0.108
[Epoch 2, Batch 600] Loss: 0.093
[Epoch 3, Batch 200] Loss: 0.078
[Epoch 3, Batch 400] Loss: 0.079
[Epoch 3, Batch 600] Loss: 0.073
[Epoch 4, Batch 200] Loss: 0.067
[Epoch 4, Batch 400] Loss: 0.071
[Epoch 4, Batch 600] Loss: 0.066
[Epoch 5, Batch 200] Loss: 0.058
[Epoch 5, Batch 400] Loss: 0.058
[Epoch 5, Batch 600] Loss: 0.057
[Epoch 6, Batch 200] Loss: 0.046
[Epoch 6, Batch 400] Loss: 0.052
[Epoch 6, Batch 600] Loss: 0.050
[Epoch 7, Batch 200] Loss: 0.046
[Epoch 7, Batch 400] Loss: 0.046
[Epoch 7, Batch 600] Loss: 0.045
[Epoch 8, Batch 200] Loss: 0.037
[Epoch 8, Batch 400] Loss: 0.036
[Epoch 8, Batch 600] Loss: 0.048
[Epoch 9, Batch 200] Loss: 0.039
[Epoch 9, Batch 400] Loss: 0.039
[Epoch 9, Batch 600] Loss: 0.036
[Epoch 10, Batch 200] Loss: 0.030
[Epoch 10, Batch 400] Loss: 0.035
[Epoch 10, Batch 600] Loss: 0.033
Validat

# Predict Test Data

In [9]:
def predict_test():
    model.eval()
    predictions = []
    with torch.no_grad():
        for images in test_loader:
            images = images.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            predictions.extend(predicted.cpu().numpy())
    
    submission = pd.DataFrame({
        'ImageId': range(1, len(predictions) + 1),
        'Label': predictions
    })
    submission.to_csv('submission.csv', index=False)
    print("Predictions saved to submission.csv")

In [10]:
predict_test()

Predictions saved to submission.csv
